# KC-ML2 TAKE-HOME PROJECT

## Show and Tell: A Neural Image Caption Generator

# Simple paper review
## 1. Introduction
#### Task: Automatically describing the content of an image
__Visual understanding__ 분야는 기존의 Computer vision 문제들, Image classification 혹은 Object recognition 보다 어렵다.  
이미지를 문장으로 설명해내는 Task는 이미지에 포함된 객체와 함께 그것들의 특성과 행동까지 잡아내야 하며, 이를 자연어로 표현해야 하는데, 이는 언어모델까지 필요하다.  
저자들은 이 Task를 풀기 위해 __Machine translation__으로부터 영감을 얻었다.  
문장 내의 단어들을 해석, 치환, 재배치 하는 기존의 방식에서, 최근 __RNN__을 이용한 __Encoder-Decoder__ 방식이 훨씬 간단하고 좋은 성능으로 해결하고 있다.  
__Encoder-RNN__이 문장을 적절한 __Vector representation__으로 변환하고, __Decoder-RNN__이 __vector__ 로부터 번역한 문장을 생성한다.  
저자들은 이 __Nerual Image Caption__(or __NIC__)라 불리는 __Encoder-RNN__을 __Encoder-CNN__으로 대체한 모델을 제안한다.

Figure 1. 제안한 NIC model의 개요   
![Figure1](resources/figure1.png "figure1")

#### Proposed Model: CNN encoder and RNN decoder(like machine translation), NIC
__CNN__은 기존의 Computer vision 문제들에서 우수한 성능을 내고 있고, 이는 __CNN__이 이미지를 잘 __embeddeing__ 한다는 것에 설득력이 있다.  
따라서 __CNN__을 이미지 encoder로 사용하는 것은 자연스러운 일이고, 저자들은 __ImageNet__으로 __pre-trained__ 된 모델을 사용했다.  
__NIC__는 다른 __NN__과 같이 SGD를 통해 학습된다.  
__NIC__는 __Pascal, Flicker8k, 30k, MSCOCO, SBU dataset__에서 사람과 근접한 __State of the art__를 달성했다.

## 3. Model
__Machine translation__과 마찬가지로 __Encoder__는 고정된 차원의 __vector__로 __encoding__ 하고,  
__Decoder__는 해당 __vector__를 __decoding__하여 이미지를 설명하는 문장을 생성한다.  
당시 연구들은 __sequence model__이 주어졌을 때 __correct translation__의 확률을 __maximize__하는 방향으로 학습하는 것이 좋다고 통계적으로 보여진다.  
저자들은 아래와 같은 수식을 __Maximizing__하는 방식으로 모델을 학습시켰다.

Equation 1. Image __*I*__ 가 주어졌을 때, Description __*S*__  
![Eqation1](resources/equation1.png "equation1")

문장 __*S*__ 는 길이가 제한적이지 않고, 따라서 __joint probability__를 이용한 아래 수식처럼 표현할 수 있다.

Equation 2. Joint probability  
![Eqation2](resources/equation2.png "equation2")

문장을 생성하는 모델에 __RNN__을 사용하는것은 자연스럽고, 저자들은 RNN을 더 Concrete하게 만들기 위해 두 가지 Crucial한 선택을 했다.  

#### 1. 어떤 Non-linear function을 써야 학습이 잘 될 것인가.  
이에 대한 문제를 __LSTM__을 사용하여 해결하였다.  
__LSTM__은 __Vanishing or exploding gradient__ 문제를 잘 해결하기 때문에 당시 __sequence task__에서 __State of the art__를 달성하였고, 저자들도 이를 선택했다.   

#### 2. 어떻게 이미지와 단어를 동시에 입력으로 넣어줄 수 있을까.  
2014 이미지넷에서 우승한 __GoogLeNet__을 사용하여 이미지를 __embedding__ 했다.  
기존의 Computer vision 문제를 잘 해결하는 모델이 이미지를 잘 표현하는 __vector representation__를 만들 것이라 생각했다.  
또한 __문장을 word 단위로 split__ 하여 __Image vector__와 같은 차원으로 __embedding__ 했다. 

### 3.1. LSTM based Sentence Generator
__LSTM__은 __Vanishing or exploding gradient__ 문제를 잘 해결하기 때문에 선택했고, __LSTM__의 전체적인 구조는 아래와 같다.  

Figure 2. LSTM structure  
![Figure2](resources/figure2.png "figure2")

__Encoder-CNN__과 결합한 LSTM의 모습은 아래와 같고, 모든 LSTM의 parameter는 공유된다.

Figure 3. LSTM model combined with a CNN image embedder  
![Figure3](resources/figure3.png "figure3")

이미지는 맨 처음 입력 단 한번만 들어가고, 이미지 벡터로부터 LSTM이 출력한 결과를 다음 LSTM의 입력으로 넣으면서 학습, 추론한다.  
저자들은 매 step마다 이미지를 넣어주는 시도를 했으나 이는 오히려 더 쉽게 Overfit 되는 결과를 보였다.  

#### Inference
저자들은 __NIC__에서 __Inference__하는 두 가지 방법을 제시했다.  

##### Sampling
__Sampling__ 방식은 아주 단순하다.  
최대 문장의 길이가 될 때 혹은 끝나는 신호가 나올 때까지 __LSTM__에서 __Softmax__를 거쳐 출력된 최대 값의 단어를 이어붙여, 이를 다음 LSTM의 입력으로 넣어주는 방식이다.

##### BeamSearch
__BeamSearch__는 매 t번째까지의 입력으로 만들어진 문장 k개를 유지하며, k개의 문장들로부터 t+1번째까지의 문장 중 다시 k개를 반환하는 방식이다.  
k=1 일 때는 굉장히 Greedy하며 BLEU score는 k=20일 때의 BLEU score보다 평균적으로 2점 정도 하락했다.  
 
## 4. Experiments
### 4.1 Evaluation Metrics
__Image description__에서 가장 많이 사용되는 __Metric__은 __BLEU score__이고, __n-gram__을 통해 평가된다.  
__n-gram__이란 다음에 나올 단어를 예측할 때 은 앞선 __n-1__개의 단어에 의존하는 방식이다.  
주로 __1-gram__을 많이 이용하고 저자들 또한 __BLEU-1__을 주로 이용하였고, 추가적으로 __ME-TEOR, Cider score__도 제시하였다.

### 4.2. Datasets

다음과 같은 Dataset을 이용하였다.  

Table 1. Datasets.  
![table1](resources/table1.png "table1")

__SBU__를 제외하고는 __모두 5개의 문장__이 __Labeling__ 되어 있다.  
저자들은 __SBU__가 __Flikcr__에서 사용자들이 올린 Description이기 때문에 Noise가 있다고 보았다.  
또한 __Pascal__은 Test를 위해서만 사용하였는데, 나머지 4개의 Data로 학습을 하고 Testing 하였다.  

### 4.3. Result
#### 4.3.1. Training Details

논문에서 저자들은 아래와 같은 사항들을 이용하여 실험했다.

##### 1. 학습 시 __CNN__은 __Imagnet__을 통해 __pre-trained__ 된 __weight__를 그대로 이용하였고, __fine tuning__은 하지 않았다.  
##### 2. __Word embedding vector__도 __pre-trained__ 된 모델을 써 보았으나 효과가 없었다.  
##### 3. __SGD__로 학습했고 __fixed learning rate__를 사용, __decay__는 사용하지 않았다.  
##### 4. __Word embedding size__와 LSTM의 크기는 512로 셋팅했다.  
##### 5. __Overfitting__을 피하기 위해 __Dropout__과 __ensemble__을 사용했다. - BLEU score 향상은 거의 없었다.  
##### 6. __Hidden layer__의 개수와 깊이를 다양하게 설정했다.  

#### 4.3.2. Generation Results

Table 2. BLEU-1 score.  
![table2](resources/table2.png "table2")

사용한 4가지 Datasets 전부에서 __SOTA BLEU-1 score__를 갱신했다.  

#### 4.3.3. Transfer Learning, Data Size and Label Quality

어떤 __datasets__로부터 학습된 모델은 다른 __datasets__에도 적용될 수 있는지 실험했다.  
같은 유저 집단이 만든 __Flickr dataset__에서는 __Transfer learning__이 효과가 있었다.  
__Flickr30k__로 학습된 모델을 이용하면 __Flickr8k__에서 __BLEU score__가 4점 정도 상승한다.  
__MSCOCO__는 __Flickr30k__ 보다 5배 크지만, __dataset 간 __miss-match__가 많았고 __BLEU score__가 10점 정도 내려갔다.  

#### 4.3.4. Generation Diversity Discussion

저자들은 __Generate model__ 관점에서, 얼마나 다양한 문장들을 생성할 수 있는지, 얼마나 수준이 높은지를 확인하였다.  
__k=20__인 __BeamSearch__에서 상위 15개 정도는 __BLEU-1 score__ 기준으로 사람과 견줄만한 __58__점 정도를 달성했다.  

Table 3. MSCOCO test set의 생성된 몇가지 예시.  
![table3](resources/table3.png "table3")

#### 4.3.7. Analysis of Embeddings
__one-hot encoding__과 다르게 __Embedding__은 __Word dictionary__의 크기에 제한되지 않는다.  
저자들은 __Embedding space__에서 __KNN__을 이용한 몇 가지 예시를 제시했는데 아래와 같다.  

Table 6. KNN을 이용한 Word embedding space analysis.  
![table6](resources/table6.png "table6")

__Embedding Vector__는 다른 __Vision component__에도 도움을 줄 수 있는데,  
__horse__와 __pony, donkey__는 근접하기 때문에 __CNN__이 __horse-looking__ 동물의 특징을 추출하는 것이 더 수월해질 것이다.  
아주 심한 경우에 __Unicorn__ 같은 몇몇 예시들도 __horse__와 근접하기 때문에, 기존의 __bag of word__ 방식보다 더 많은 정보를 줄 수 있다.


# Data processing

In [1]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg19 import VGG19, preprocess_input

import numpy as np
import h5py
import string
import pickle

from os import listdir
from os.path import join, isdir, isfile, exists

Using TensorFlow backend.


In [2]:
meta_info = {
    'projcet_name': 'kc-ml2 Take-home project',
    'image_dir': 'Flicker8k_Dataset/',
    'train_list': 'Flickr8k_text/Flickr_8k.trainImages.txt',
    'dev_list': 'Flickr8k_text/Flickr_8k.devImages.txt',
    'test_list': 'Flickr8k_text/Flickr_8k.testImages.txt',
    'text_dir': 'Flickr8k_text/'
}

print(meta_info['projcet_name'])
print(listdir(meta_info['image_dir'])[:5])

kc-ml2 Take-home project
['3342309960_c694b2cce9.jpg', '3234375022_1464ea7f8a.jpg', '1821238649_2fda79d6d7.jpg', '3177468217_56a9142e46.jpg', '337647771_3b819feaba.jpg']


## Image preprocessing

In [3]:
""" feature extract CNN model
This paper used GoogLeNet (InceptionV1) which got good grades in ImageNet 2014
but for convenience of implementation, I used various models including InceptionV3 in built-in module of keras.
My model has the best performance at VGG19.
"""
def model_select(model_name):
    if model_name == 'VGG16':
        from keras.applications.vgg16 import VGG16, preprocess_input
        model = VGG16() # 4096
    elif model_name == 'VGG19':
        from keras.applications.vgg19 import VGG19, preprocess_input 
        model = VGG19() # 4096
    elif model_name == 'ResNet50':
        from keras.applications.resnet50 import ResNet50, preprocess_input
        model = ResNet50() # 4096
    elif model_name == 'InceptionV3':
        from keras.applications.inception_v3 import InceptionV3, preprocess_input
        model = InceptionV3() # 2048,
    elif model_name == 'InceptionResNetV2':
        from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
        model = InceptionResNetV2() # 1536,
    return model

In [4]:
model_name = 'VGG19'
base_model = model_select(model_name)
# using FC2 layer output
cnn_model = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)
cnn_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

### Image to feature

In [5]:
"""
Usually training set is the bigger,
so I prefer to testing with validation set first.
"""

dev_features = {}
dev_h5 = 'dev_features.h5'
with h5py.File(dev_h5, 'w') as h5f:
    with open(meta_info['dev_list']) as f:
        c = 0 # count
        contents = f.read()
        for line in contents.split('\n'):
            if line == '': # last line or error line
                print(c)
                continue
            if c % 100 == 0:
                print(c)
            # Unlike other models, inception models use the larger image sizes.
            if model_name.find('Inception') != -1:
                target_size = (299, 299)
            else:
                target_size = (224, 224)
                
            img_path = line
            img = load_img(meta_info['image_dir'] + img_path, target_size=target_size)
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img)
            feature = cnn_model.predict(img)
            h5f.create_dataset(img_path.split('.')[0], data=feature)
            c += 1

0
100
200
300
400
500
600
700
800
900
1000


In [6]:
# feature test
with h5py.File('dev_features.h5', 'r') as h5f:
    print(h5f['2090545563_a4e66ec76b'][:])
    print(h5f['2090545563_a4e66ec76b'][:].shape)

[[0.        0.        1.0180278 ... 0.        0.        0.       ]]
(1, 4096)


In [7]:
train_features = {}
train_h5 = 'train_features.h5'
with h5py.File(train_h5, 'w') as h5f:
    with open(meta_info['train_list']) as f:
        c = 0 # count
        contents = f.read()
        for line in contents.split('\n'):
            if line == '': # last line or error line
                print(c)
                continue
            if c % 1000 == 0:
                print(c)

            if model_name.find('Inception') != -1:
                target_size = (299, 299)
            else:
                target_size = (224, 224)
                
            img_path = line
            img = load_img(meta_info['image_dir'] + img_path, target_size=target_size)
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img)
            feature = cnn_model.predict(img)
            h5f.create_dataset(img_path.split('.')[0], data=feature)
            c += 1

0
1000
2000
3000
4000
5000
6000


In [8]:
test_features = {}
test_h5 = 'test_features.h5'
with h5py.File(test_h5, 'w') as h5f:
    with open(meta_info['test_list']) as f:
        c = 0 # count
        contents = f.read()
        for line in contents.split('\n'):
            if line == '': # last line or error line
                print(c)
                continue
            if c % 100 == 0:
                print(c)

            if model_name.find('Inception') != -1:
                target_size = (299, 299)
            else:
                target_size = (224, 224)
                
            img_path = line
            img = load_img(meta_info['image_dir'] + img_path, target_size=target_size)
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img)
            feature = cnn_model.predict(img)
            h5f.create_dataset(img_path.split('.')[0], data=feature)
            c += 1

0
100
200
300
400
500
600
700
800
900
1000


## Text preprocessing

In [9]:
""" full captions to dictionary
The dictionary has full dataset(training, validation, and test captions), 
and numbers are eliminated from all captions.
Removing numbers improves performance (by about 3 points for bleu-1)
"""

captions = dict()
words = set()

with open(join(meta_info['text_dir'], 'Flickr8k.token.txt')) as f:
    contents = f.read()
    n_captions = 0
    for line in contents.split('\n'):
        if line == '':
            print(n_captions)
            continue
        if n_captions % 10000 == 0:
            print(n_captions)
        
        file, caption = line.split('\t')
        
        table = str.maketrans('', '', string.punctuation)
        
        caption2 = []
        for word in caption.split():
            # remove number
            if word.isalpha():
                caption2.append(word.translate(table))
        caption = ' '.join(caption2)
        
        img_id = file.split('.')[0]
        
        if img_id in captions.keys():
            captions[img_id].append(caption)
        else:
            captions[img_id] = [caption]
        n_captions += 1

        [words.add(word) for word in caption.split()]
        
print('number of images: %d' % len(captions))
print('number of catpions: %d' % n_captions)
print('number of words: %d' % len(words))

0
10000
20000
30000
40000
40460
number of images: 8092
number of catpions: 40460
number of words: 9068


In [10]:
# train set caption test
print(captions['2513260012_03d33305cf'])
# dev set caption test
print(captions['2090545563_a4e66ec76b'])
# test set caption test
print(captions['3385593926_d3e9c21170'])

['A black dog is running after a white dog in the snow', 'Black dog chasing brown dog through snow', 'Two dogs chase each other across the snowy ground', 'Two dogs play together in the snow', 'Two dogs running through a low lying body of water']
['the boy laying face down on a skateboard is being pushed along the ground by another boy', 'Two girls play on a skateboard in a courtyard', 'Two people play on a long skateboard', 'Two small children in red shirts playing on a skateboard', 'two young children on a skateboard going across a sidewalk']
['The dogs are in the snow in front of a fence', 'The dogs play on the snow', 'Two brown dogs playfully fight in the snow', 'Two brown dogs wrestle in the snow', 'Two dogs playing in the snow']


In [11]:
""" Only dev captions are taken from the full captions set.
Unlike above caption, this captions has sign of start and end for sequence.
Each sssss, eeeee, but <> are not included in order to simplify the processing of the tokenizer.
keras' tokenizer removes <>, so need to further processing in this process.
"""

dev_captions = dict()
dev_words = set()

with open(join(meta_info['text_dir'], 'Flickr_8k.devImages.txt')) as f:
    contents = f.read()
    n_dev_captions = 0
    for line in contents.split('\n'):
        if line == '':
            print(n_dev_captions)
            continue
        if n_dev_captions % 10000 == 0:
            print(n_dev_captions)
        
        file = line.split('.')[0]
        
        for caption in captions[file]:
            # start sign: sssss
            # end sign: eeeee
            caption = 'sssss ' + caption + ' eeeee'
            caption = caption.replace('\n', '')
            
            if file in dev_captions.keys():
                dev_captions[file].append(caption)
            else:
                dev_captions[file] = [caption]
            n_dev_captions += 1
            
            [dev_words.add(word) for word in caption.split()]

print('number of catpions: %d' % len(dev_captions))
print('number of catpions: %d' % n_dev_captions)
print('number of words: %d' % len(dev_words))

0
5000
number of catpions: 1000
number of catpions: 5000
number of words: 3409


In [12]:
# dev set caption test
print(dev_captions['2090545563_a4e66ec76b'])

['sssss the boy laying face down on a skateboard is being pushed along the ground by another boy eeeee', 'sssss Two girls play on a skateboard in a courtyard eeeee', 'sssss Two people play on a long skateboard eeeee', 'sssss Two small children in red shirts playing on a skateboard eeeee', 'sssss two young children on a skateboard going across a sidewalk eeeee']


In [13]:
"""
Unlike a dev set, training set must count the maximum number of words in single sentence.
Variable M do that role.
"""

train_captions = dict()
train_words = set()

M = 0 # max length in single sentence

with open(join(meta_info['text_dir'], 'Flickr_8k.trainImages.txt')) as f:
    contents = f.read()
    n_train_captions = 0
    for line in contents.split('\n'):
        if line == '':
            print(n_train_captions)
            continue
        if n_train_captions % 10000 == 0:
            print(n_train_captions)
        
        file = line.split('.')[0]
        
        for caption in captions[file]:
            caption = 'sssss ' + caption + ' eeeee'
            caption = caption.replace('\n', '')
            
            if file in train_captions.keys():
                train_captions[file].append(caption)
            else:
                train_captions[file] = [caption]
            n_train_captions += 1
            
            t = caption.split()
            if len(t) > M:
                M = len(t)
            [train_words.add(word) for word in t]

# n_vocabs = len(train_words) # all word, based str.split()

print('number of catpions: %d' % len(train_captions))
print('number of catpions: %d' % n_train_captions)
print('number of words: %d' % len(train_words))

# print('vocabulary size: %d' % n_vocabs)
print('max number of words in single sentence: %d' % M)

0
10000
20000
30000
number of catpions: 6000
number of catpions: 30000
number of words: 7816
max number of words in single sentence: 36


In [14]:
# train set caption test
print(train_captions['2513260012_03d33305cf'])

['sssss A black dog is running after a white dog in the snow eeeee', 'sssss Black dog chasing brown dog through snow eeeee', 'sssss Two dogs chase each other across the snowy ground eeeee', 'sssss Two dogs play together in the snow eeeee', 'sssss Two dogs running through a low lying body of water eeeee']


In [15]:
test_captions = dict()
test_words = set()

with open(join(meta_info['text_dir'], 'Flickr_8k.testImages.txt')) as f:
    contents = f.read()
    n_test_captions = 0
    for line in contents.split('\n'):
        if line == '':
            print(n_test_captions)
            continue
        if n_test_captions % 10000 == 0:
            print(n_test_captions)
        
        file = line.split('.')[0]
        
        for caption in captions[file]:
            caption = 'sssss ' + caption + ' eeeee'
            caption = caption.replace('\n', '')
            
            if file in test_captions.keys():
                test_captions[file].append(caption)
            else:
                test_captions[file] = [caption]
            n_test_captions += 1
            
            [test_words.add(word) for word in caption.split()]

print('number of catpions: %d' % len(test_captions))
print('number of catpions: %d' % n_test_captions)
print('number of words: %d' % len(test_words))

0
5000
number of catpions: 1000
number of catpions: 5000
number of words: 3266


In [16]:
# test set caption test
print(test_captions['3385593926_d3e9c21170'])

['sssss The dogs are in the snow in front of a fence eeeee', 'sssss The dogs play on the snow eeeee', 'sssss Two brown dogs playfully fight in the snow eeeee', 'sssss Two brown dogs wrestle in the snow eeeee', 'sssss Two dogs playing in the snow eeeee']


In [17]:
""" make tokenizer using keras.
Making tokenizer, only use train captions.
"""
def make_tokenizer(captions):
    texts = []
    for _, caption_list in captions.items():
        for caption in caption_list:
            texts.append(caption)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    return tokenizer

In [18]:
tokenizer = make_tokenizer(train_captions)
n_vocabs = len(tokenizer.word_index) + 1 # because index 0, plus 1
print('number of vocabulary: %d' % n_vocabs)
# print(tokenizer.word_index)

number of vocabulary: 7277


In [19]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
# print(len(tokenizer.word_index))

In [21]:
""" Make sequence, Make next word based ground truth.
If single sentence consisting of N words, N + 1(because nd sign) sequences are created.
Ex) Hi, I am a boy.
sequence                 -> next word
[]   []   []   []   [Hi] -> I
[]   []   []   [Hi] [I]  -> am
[]   []   [Hi] [I]  [am] -> a
...
[Hi] [I] [am] [a] [boy] -> eeeee(end sign)
"""
train_sequences = list()
train_next_word = list()

c = 0
train_sequences_h5 = 'train_sequences.h5'
train_next_word_h5 = 'train_next_word.h5'
h5f1 = h5py.File(train_sequences_h5, 'w')
h5f2 = h5py.File(train_next_word_h5, 'w')
for img_id, captions in train_captions.items():
#     print(img_id)
    Xtrain = list()
    ytrain = list()
    for caption in captions:
        sequence = tokenizer.texts_to_sequences([caption])[0]
        
        for i in range(1, len(sequence)): # except start sign
            if c % 100000 == 0:
                print(c)
            train_sequences.append(pad_sequences([sequence[:i]], M)[0])
            Xtrain.append(pad_sequences([sequence[:i]], M)[0])
            train_next_word.append(to_categorical([sequence[i]], num_classes=n_vocabs)[0])
            ytrain.append(to_categorical([sequence[i]], num_classes=n_vocabs)[0])
            c += 1
    h5f1.create_dataset(img_id, data=Xtrain)
    h5f2.create_dataset(img_id, data=ytrain)
h5f1.close()
h5f2.close()
print(c)

0
100000
200000
300000
352425


In [22]:
# test sequences and next word
print(train_sequences[0])
print(train_next_word[0])
print(train_sequences[1])
print(train_next_word[1])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
[0. 1. 0. ... 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1]
[0. 0. 0. ... 0. 0. 0.]


In [23]:
dev_sequences = list()
dev_next_word = list()

c = 0
dev_sequences_h5 = 'dev_sequences.h5'
dev_next_word_h5 = 'dev_next_word.h5'
h5f1 = h5py.File(dev_sequences_h5, 'w')
h5f2 = h5py.File(dev_next_word_h5, 'w')
for img_id, captions in dev_captions.items():
#     print(img_id)
    Xdev = list()
    ydev = list()
    for caption in captions:
        text = tokenizer.texts_to_sequences([caption])[0]
        
        for i in range(1, len(text)):
            if c % 10000 == 0:
                print(c)
            dev_sequences.append(pad_sequences([text[:i]], M)[0])
            Xdev.append(pad_sequences([text[:i]], M)[0])
            dev_next_word.append(to_categorical([text[i]], num_classes=n_vocabs)[0])
            ydev.append(to_categorical([text[i]], num_classes=n_vocabs)[0])
            c += 1
    h5f1.create_dataset(img_id, data=Xdev)
    h5f2.create_dataset(img_id, data=ydev)
h5f1.close()
h5f2.close()
print(c)

0
10000
20000
30000
40000
50000
58661


In [24]:
test_sequences = list()
test_next_word = list()

c = 0
test_sequences_h5 = 'test_sequences.h5'
test_next_word_h5 = 'test_next_word.h5'
h5f1 = h5py.File(test_sequences_h5, 'w')
h5f2 = h5py.File(test_next_word_h5, 'w')
for img_id, captions in test_captions.items():
#     print(img_id)
    Xtest = list()
    ytest = list()
    for caption in captions:
        text = tokenizer.texts_to_sequences([caption])[0]
        
        for i in range(1, len(text)):
            if c % 10000 == 0:
                print(c)
            test_sequences.append(pad_sequences([text[:i]], M)[0])
            Xtest.append(pad_sequences([text[:i]], M)[0])
            test_next_word.append(to_categorical([text[i]], num_classes=n_vocabs)[0])
            ytest.append(to_categorical([text[i]], num_classes=n_vocabs)[0])
            c += 1
    h5f1.create_dataset(img_id, data=Xtest)
    h5f2.create_dataset(img_id, data=ytest)
h5f1.close()
h5f2.close()
print(c)

0
10000
20000
30000
40000
50000
58389


# Data processing end.

## Bellow code isn't need to look.

# h5 -> Pickle

In [ ]:
train_sequences = list()
train_next_word = list()

c = 0
train_sequences_pkl = 'train_sequences.pkl'
train_next_word_pkl = 'train_next_word.pkl'

X = dict()
Y = dict()

for img_id, captions in train_captions.items():
#     print(img_id)
    Xtrain = list()
    ytrain = list()
    for caption in captions:
        text = tokenizer.texts_to_sequences([caption])[0]
        
        for i in range(1, len(text)):
            if c % 100000 == 0:
                print(c)
            train_sequences.append(pad_sequences([text[:i]], M)[0])
            Xtrain.append(pad_sequences([text[:i]], M)[0])
            train_next_word.append(to_categorical([text[i]], num_classes=n_vocabs)[0])
            ytrain.append(to_categorical([text[i]], num_classes=n_vocabs)[0])
            c += 1
    X[img_id] = Xtrain
    Y[img_id] = ytrain
with open(train_sequences_pkl, 'wb') as f:
    pickle.dump(X, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(train_next_word_pkl, 'wb') as f:
    pickle.dump(Y, f, protocol=pickle.HIGHEST_PROTOCOL)
print(c)

In [ ]:
with open(train_sequences_pkl, 'rb') as f:
    test = pickle.load(f)
    print(test['2513260012_03d33305cf'])

# not needed

In [23]:
train_id_word = dict()

for i, word in enumerate(train_words):
    train_id_word[i] = word
    train_word_id[word] = i

print(len(train_id_word))
print(len(train_word_id))

8256
8256


In [24]:
dev_id_word = dict()
dev_word_id = dict()

for i, word in enumerate(dev_words):
    dev_id_word[i] = word
    dev_word_id[word] = i

print(len(dev_id_word))
print(len(dev_word_id))

3523
3523


In [10]:
sequences = list()
nextwords = list()

data = {}
for captions in train_captions.items():
#     print(captions)
    data[captions[0]] = []
    for caption in captions[1]:
        t = []
        for word in caption.split():
            t.append(train_word_id[word])
        data[captions[0]].append(t)
#     print(data)
print(len(data))

6000


In [11]:
id_seq = {}
id_y = {}
c = 0
for key, value in data.items():
    sub_seqs = []
    Y = []
    for seq in value:
        for i in range(1, len(seq)):
            if c % 100000 == 0:
                print(c)
            sub_seqs.append(sequence.pad_sequences([seq[:i]], max_length)[0])
            y = to_categorical([seq[i]], num_classes=n_vocab + 1)
            Y.append(y[0])
            c += 1
            
    id_seq[key] = sub_seqs
    id_y[key] = Y
print(c)
#         print(id_seq)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
383454


In [12]:
h5file_path = 'train_id_seq.h5'
with h5py.File(h5file_path, 'w') as h5f:
    for key, value in id_seq.items():
        h5f.create_dataset(key, data=value)
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

In [13]:
h5file_path = 'train_id_seq.h5'
with h5py.File(h5file_path, 'r') as h5f:
    print(h5f['667626_18933d713e'][:])
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

[[8133    0    0 ...    0    0    0]
 [8133 4381    0 ...    0    0    0]
 [8133 4381  850 ...    0    0    0]
 ...
 [8133 4752 3548 ...    0    0    0]
 [8133 4752 3548 ...    0    0    0]
 [8133 4752 3548 ...    0    0    0]]


In [15]:
h5file_path = 'train_id_y.h5'
with h5py.File(h5file_path, 'w') as h5f:
    for key, value in id_y.items():
        h5f.create_dataset(key, data=value)
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

In [16]:
h5file_path = 'train_id_y.h5'
with h5py.File(h5file_path, 'r') as h5f:
    print(h5f['667626_18933d713e'][:])
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
sequences = list()
nextwords = list()

data = {}
for captions in dev_captions.items():
#     print(captions)
    data[captions[0]] = []
    for caption in captions[1]:
        t = []
        for word in caption.split():
            t.append(dev_word_id[word])
        data[captions[0]].append(t)
#     print(data)
print(len(data))

1000


In [11]:
id_seq = {}
id_y = {}
c = 0
for key, value in data.items():
    sub_seqs = []
    Y = []
    for seq in value:
        for i in range(1, len(seq)):
            if c % 10000 == 0:
                print(c)
            sub_seqs.append(sequence.pad_sequences([seq[:i]], max_length, padding='post')[0])
            y = to_categorical([seq[i]], num_classes=n_vocab)
            Y.append(y[0])
            c += 1
    id_seq[key] = sub_seqs
    id_y[key] = Y
print(c)
#         print(id_seq)

0
10000
20000
30000
40000
50000
60000
64445


In [12]:
h5file_path = 'dev_id_seq.h5'
with h5py.File(h5file_path, 'w') as h5f:
    for key, value in id_seq.items():
        h5f.create_dataset(key, data=value)
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

In [13]:
h5file_path = 'dev_id_seq.h5'
with h5py.File(h5file_path, 'r') as h5f:
    print(h5f['2090545563_a4e66ec76b'][:])
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

[[1626    0    0 ...    0    0    0]
 [1626 1622    0 ...    0    0    0]
 [1626 1622  830 ...    0    0    0]
 ...
 [1626 3127 2829 ...    0    0    0]
 [1626 3127 2829 ...    0    0    0]
 [1626 3127 2829 ...    0    0    0]]


In [14]:
h5file_path = 'dev_id_y.h5'
with h5py.File(h5file_path, 'w') as h5f:
    for key, value in id_y.items():
        h5f.create_dataset(key, data=value)
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

In [15]:
h5file_path = 'dev_id_y.h5'
with h5py.File(h5file_path, 'r') as h5f:
    print(h5f['2090545563_a4e66ec76b'][:])
# print(feature_np)
# np.squeeze(feature_np)
# print(feature_np.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
